Ce notebook est dédié au classifier (Lyrics to genre) du projet NLP.
Commençons par importer les différents packages

In [ ]:
!pip install kaggle
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import random
import nltk
import tensorflow as tf
import matplotlib.pyplot as plt
import tiktoken
import os

#from google.colab import userdata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from spacy.lang.en.stop_words import STOP_WORDS as en_stop


Ensuite vous aurez besoin de récupérer le dataset en provenance de kaggle.
Pour se faire, je vous prie de suivre ce tuto
https://www.kaggle.com/discussions/general/74235#2580958


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
!kaggle datasets download -d carlosgdcj/genius-song-lyrics-with-language-information
!unzip genius-song-lyrics-with-language-information.zip

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


100% 3.04G/3.04G [01:18<00:00, 41.6MB/s]
100% 3.04G/3.04G [01:18<00:00, 41.7MB/s]
Archive:  genius-song-lyrics-with-language-information.zip
  inflating: song_lyrics.csv         


Etape de préprocessing 'rapide'.

In [95]:
# n = 100 every 100th line = 1% of the lines 50 000 lines taken
df = pd.read_csv("../song_lyrics.csv", skiprows=lambda i: i % 500 != 0)
print(df.index)
df = df[df['tag'] != 'misc']
if 'language' in df.columns:
    df = df[df['language'] == 'en']
df = df[['title', 'lyrics', 'tag']]
df.reset_index(drop=True, inplace=True)
# To shuffle randomnly datas
df = df.sample(frac = 1)
# Split the data into features (X) and labels (Y)
X = df['lyrics']
Y = df['tag']

# Split the data into training and test sets (80% training, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

RangeIndex(start=0, stop=10269, step=1)


Utilisons les tokenizers vu en cours encore une fois dans un soucis de rapidité

In [25]:
#Define tokenizers from course
def lemma_tokenize(doc):
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(t) for t in word_tokenize(doc)]

def char_tokenize(doc):
    return [char for char in doc]

def byte_tokenize(doc):
    tokens = doc.encode("utf-8")
    tokens = list(map(int, tokens))
    return [str(token) for token in tokens]

def gpt_tokenize(doc):
    enc = tiktoken.encoding_for_model("gpt-4")
    tokens = enc.encode(doc)
    return [str(token) for token in tokens]

Création du classifier en utilisant scikit-learn
On peut s'amuser à jouer sur les différents paramètres et hyperparamètres pour voir si on obtient de meilleur résultats

In [96]:
# Create model, we can test them one by one or even customize them using hyperparameters tunning
model = make_pipeline(CountVectorizer(ngram_range = (1,1), stop_words = en_stop), MultinomialNB()) #Naive Bayes
#model = make_pipeline(CountVectorizer(ngram_range = (1,1), stop_words = en_stop), LogisticRegression()) #Logistic Regression
model.fit(X_train, Y_train)


NameError: name 'MultinomialNB' is not defined

On évalue dans un premier temps le modèle simplement en regardant sa précision.

In [ ]:
accuracy = model.score(X_test, Y_test)
print("Accuracy:", accuracy)

Ce-dessous une liste de toutes les combinaisons qui ont été testées une par une. Pour obtenir une vue des résultats : voir l'annexe dans le repo github.

In [ ]:
#model = make_pipeline(CountVectorizer(tokenizer=gpt_tokenize, ngram_range=(1, 1)), scaler, LogisticRegression( max_iter = 1000, solver='saga',penalty='l2'))
#model = make_pipeline(CountVectorizer(tokenizer=byte_tokenize, ngram_range=(1, 1)), scaler, LogisticRegression( max_iter = 1000, solver='saga',penalty='l2'))
#model = make_pipeline(CountVectorizer(tokenizer=word_tokenize, ngram_range=(1, 1)), scaler, LogisticRegression( max_iter = 1000, solver='saga',penalty='l2'))
#model = make_pipeline(CountVectorizer(ngram_range=(1, 1), scaler, LogisticRegression( max_iter = 3000, solver='lbfgs'))
#model = make_pipeline(CountVectorizer(ngram_range=(1, 2), scaler, LogisticRegression( max_iter = 3000, solver='lbfgs'))


#model = make_pipeline(CountVectorizer(tokenizer=word_tokenize, ngram_range=(1, 1)), scaler, LogisticRegression( max_iter = 1000, solver='saga',penalty='elasticnet', l1-ratio=0.5))


Voici comment évaluer le modèle de manière plus pertinente.
On se permet d'utiliser

*   La matrice de confusion
*   Le classification report fourni par sk-learn
*   L'évaluation empirique

Matrice de confusion : on voit précisément où le modèle s'est trompé / où il a bien prédit



In [ ]:
# Print classification report and confusion matrix
print("Classification Report:\n", classification_report(Y_test, Y))
cm = confusion_matrix(Y_test, Y)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
plt.show()

Classification report : qu'on envoit vers un fichier excel pour ensuite le comparer avec d'autres itérations

In [ ]:
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(Y_test, y_pred))
# Convert classification report to dictionary
report_dict = classification_report(Y_test, y_pred, output_dict=True)

# Convert the dictionary to a DataFrame
df_excel = pd.DataFrame(report_dict).transpose()

# Convert the DataFrame to Excel format
df_excel.to_excel("classification_report.xlsx")

Tests empirique 'visuels' : on prend au hasard certaines musiques.
Et on regarde si notre modèle performe bien dessus ou pas.
Enfin on affiche les erreurs/réussites

In [ ]:
df_test = pd.read_csv("song_lyrics.csv", skiprows=lambda i: i % 977 != 0 , nrows=10) # Change here to test different values

df_test = df_test[df_test['tag'] != 'misc']
if 'language' in df_test.columns:
    df_test = df_test[df_test['language'] == 'en']
df_test = df_test[['title', 'lyrics', 'tag']]
df_test.reset_index(drop=True, inplace=True)

for song_name, song_lyrics, song_tag in zip(df_test['title'], df_test['lyrics'], df_test['tag']):
    print("Song:", song_name)
    print("Tag:", song_tag)
    # Convert the lyrics to a list and predict probabilities
    probabilities = model.predict_proba([song_lyrics])

    # Print the distribution of probabilities
    print("Distribution of Probabilities:")
    for class_label, probability in zip(model.classes_, probabilities[0]):
        if(probability > 0.0001):
          print(f"{class_label}: {probability:.4f}")
    max_prob_index = probabilities.argmax()
    predicted_class = model.classes_[max_prob_index]
    if predicted_class != song_tag:
        print(f'Model failed to predict. Actual tag is {song_tag}, predicted tag is {predicted_class}')
    print()

Dans une optique de performance nous avons décidé de voir ce si les perfomances étaient meilleures sans utiliser scikit-learn

Passons donc maintenant à l'implementation du classifer à l'aide de PyTorch.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import LabelEncoder


In [97]:

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


Y_train_vec = vectorizer.fit_transform(Y_train)
Y_test_vec = vectorizer.transform(Y_test)

In [98]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_vec.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_vec.toarray(), dtype=torch.float32)

label_encoder = LabelEncoder()
Y_train_indices = label_encoder.fit_transform(Y_train)
Y_test_indices = label_encoder.transform(Y_test)

Y_train_tensor = torch.tensor(Y_train_indices, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test_indices, dtype=torch.long)

In [103]:
# Define logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        self.softmax = nn.Softmax(dim=1)
        #self.relu = nn.ReLU()

    def forward(self, x):
        out = self.linear(x)
        #out = self.relu(out)
        out = self.softmax(out)
        #out = torch.sigmoid(out)
        return out

input_size = X_train_tensor.shape[1]
num_classes = len(Y_train.unique())


On utilise la CrossEntropy Loss et l'optimizer Adam

In [104]:
# Initialize the model
model = LogisticRegression(input_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [105]:
print(X_train_tensor.shape)
print(X_test_tensor.shape)
print(Y_train_tensor.shape)

torch.Size([5096, 35132])
torch.Size([1275, 35132])
torch.Size([5096])


In [106]:
num_epochs = 200
for epoch in range(num_epochs):
    # Training phase
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, Y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        _, predicted_val = torch.max(val_outputs, dim=1)
        num_correct = (predicted_val == Y_test_tensor).sum().item()
        accuracy = num_correct / Y_test_tensor.size(0) * 100

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

Epoch [1/200], Loss: 1.6071, Accuracy: 30.35%
Epoch [2/200], Loss: 1.6042, Accuracy: 34.35%
Epoch [3/200], Loss: 1.6014, Accuracy: 36.47%
Epoch [4/200], Loss: 1.5986, Accuracy: 38.82%
Epoch [5/200], Loss: 1.5959, Accuracy: 40.24%
Epoch [6/200], Loss: 1.5933, Accuracy: 41.10%
Epoch [7/200], Loss: 1.5906, Accuracy: 42.43%
Epoch [8/200], Loss: 1.5880, Accuracy: 43.53%
Epoch [9/200], Loss: 1.5854, Accuracy: 44.47%
Epoch [10/200], Loss: 1.5828, Accuracy: 44.47%
Epoch [11/200], Loss: 1.5802, Accuracy: 44.86%
Epoch [12/200], Loss: 1.5775, Accuracy: 45.18%
Epoch [13/200], Loss: 1.5749, Accuracy: 45.25%
Epoch [14/200], Loss: 1.5722, Accuracy: 45.49%
Epoch [15/200], Loss: 1.5695, Accuracy: 45.41%
Epoch [16/200], Loss: 1.5668, Accuracy: 45.73%
Epoch [17/200], Loss: 1.5641, Accuracy: 45.73%
Epoch [18/200], Loss: 1.5614, Accuracy: 45.88%
Epoch [19/200], Loss: 1.5587, Accuracy: 45.88%
Epoch [20/200], Loss: 1.5559, Accuracy: 46.12%
Epoch [21/200], Loss: 1.5532, Accuracy: 45.88%
Epoch [22/200], Loss: 

Evaluation du modèle

In [16]:
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

cm = confusion_matrix(Y_test_tensor.numpy(), predicted.numpy())
print(cm)
report = classification_report(Y_test_tensor.numpy(), predicted.numpy())
print(report)

[[  0  11   1   0   2]
 [  3 199  18   4  42]
 [  0  43 152   1   8]
 [  1  25  10   1   4]
 [  1  79   6   1  33]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.56      0.75      0.64       266
           2       0.81      0.75      0.78       204
           3       0.14      0.02      0.04        41
           4       0.37      0.28      0.32       120

    accuracy                           0.60       645
   macro avg       0.38      0.36      0.35       645
weighted avg       0.57      0.60      0.57       645

